In [1]:
from data.dataset import GraphDataset, DataAugmentationLevel
from environment.free import FreeEnvironment
from environment.guided import GuidedEnvironment
from data.parsers.parserValueProvider import RealValueBackend
from data.parsers.answerTemplateParser import AnswerTemplateParser
from data.parsers.systemTemplateParser import SystemTemplateParser
from data.parsers.logicParser import LogicTemplateParser
from utils.utils import AutoSkipMode

In [2]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
data = GraphDataset('resources/en/test_graph.json', 'resources/en/test_answers.json', use_answer_synonyms=True, augmentation=DataAugmentationLevel.ARTIFICIAL_ONLY, augmentation_version=2)

- Loading questions from  resources/en/generated/train_questions_v2.json
- only artificial answers
Loading augmentation answers from resources/en/generated/train_answers.json
===== Dataset Statistics =====
- files:  resources/en/test_graph.json resources/en/test_answers.json
- synonyms: True
- depth: 20  - degree: 13
- answers: 803
- questions: 800
- loaded original data: False
- loaded generated data: True


In [4]:
answerParser = AnswerTemplateParser()
logicParser = LogicTemplateParser()
sysParser = SystemTemplateParser()
valueBackend = RealValueBackend(a1_laender=data.a1_countries, data=data.hotel_costs)

In [5]:
def print_node(env):
    print(env.current_node.key, env.current_node.node_type)
    print(env.current_node.text)
    for a in env.current_node.answers:
        print(a.index, a.text)

In [ ]:
free = FreeEnvironment(data, "[SYS]", "USR", "SEP", 50, 100, 3, True, True, answerParser, sysParser, logicParser, value_backend, AutoSkipMode.NONE)

In [23]:
guided = GuidedEnvironment(data,  "[SYS]", "USR", "SEP", 50, 100, 3, True, False, answerParser, sysParser, logicParser, valueBackend, AutoSkipMode.NONE)

In [ ]:
obs1 = free.reset(0, -1, None)
print_node(free)
print(data.nodes_by_key[free.goal.goal_node_key].text)

In [ ]:
obs2, reward, done = free.step(1)
print_node(free)
print(reward, done)
print(free.last_valid_skip_transition_idx)

In [ ]:
free.episode_log

In [7]:
from environment.goal import UserGoal

In [27]:
goal_node = data.nodes_by_key[16478798353898107]
print(goal_node.text)
goal_guided = UserGoal(data=data, start_node=data.start_node, goal_node=goal_node, initial_user_utterance="Investigation term",
                        answer_parser=answerParser,system_parser=sysParser, value_backend=valueBackend )
goal_guided.path

Are you planning to extend your trip with private travel?


GoalPath(visited_nodes=[DialogNode.START(key: 0, answers: 0, questions: 0)
        - connected_node: 16348058621438633
        - text: START
        , DialogNode.QUESTION(key: 16348058621438633, answers: 6, questions: 0)
        - connected_node: None
        - text: What topic do you have questions about? You can either click on an answer from the suggested topics 
        , DialogNode.VARIABLE(key: 16370487978253782, answers: 1, questions: 0)
        - connected_node: None
        - text: How long will your research semester last?  
        , DialogNode.LOGIC(key: 16460422182910140, answers: 2, questions: 0)
        - connected_node: None
        - text: {{TRIP_LENGTH
        , DialogNode.QUESTION(key: 16353297472197738, answers: 3, questions: 0)
        - connected_node: None
        - text: Is this as part of a reallocation?  
        , DialogNode.INFO(key: 16370501863636503, answers: 0, questions: 10)
        - connected_node: 16353303012891903
        - text: As there has been no

In [28]:
obs1 = guided.reset(0,-1,None)
guided.goal = goal_guided

In [30]:
print_node(guided)

16348058621438633 NodeType.QUESTION
What topic do you have questions about? You can either click on an answer from the suggested topics or enter your own text.
0 Book a trip
1 Research semester
2 Travel Risk Management
3 Problem/emergency during a business trip
4 Forms, deadlines, and procedures
5 cancel a trip


In [31]:
obs2, reward, done = guided.step(2)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

0.03 False
0


In [32]:
print_node(guided)

16370487978253782 NodeType.VARIABLE
How long will your research semester last?  
0 {{TRIP_LENGTH=TIMESPAN}}


In [33]:

obs2, reward, done = guided.step(0)
print_node(guided)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

16370487978253782 NodeType.VARIABLE
How long will your research semester last?  
0 {{TRIP_LENGTH=TIMESPAN}}
0.01 False
0


In [34]:
print(guided.current_user_utterance)

4642


In [36]:
obs2, reward, done = guided.step(1)
print_node(guided)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

16460422136134149 NodeType.INFO
If you are traveling for more than 4 weeks and need help finding accommodations, please contact the business travel department.
0.03 False
0


In [37]:
obs2, reward, done = guided.step(0)
print_node(guided)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

16460422136134149 NodeType.INFO
If you are traveling for more than 4 weeks and need help finding accommodations, please contact the business travel department.
0.01 False
0


In [40]:
obs2, reward, done = guided.step(1)
print_node(guided)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

16353297472197738 NodeType.QUESTION
Is this as part of a reallocation?  
0 What is a reallocation/delegation?
1 Yes
2 No
0.03 False
0


In [46]:
obs2, reward, done = guided.step(0)
print_node(guided)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

16353297472197738 NodeType.QUESTION
Is this as part of a reallocation?  
0 What is a reallocation/delegation?
1 Yes
2 No
0.01 False
0


In [47]:
obs2, reward, done = guided.step(3)
print_node(guided)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

16370501863636503 NodeType.INFO
As there has been no reallocation, your research semester counts as a business trip.
0.03 False
0


In [48]:
obs2, reward, done = guided.step(0)
print_node(guided)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

16370501863636503 NodeType.INFO
As there has been no reallocation, your research semester counts as a business trip.
0.01 False
0


In [49]:
obs2, reward, done = guided.step(1)
print_node(guided)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

16353303012891903 NodeType.QUESTION
Are you traveling alone or with your family?
0 Alone
1 With family
0.03 False
0


In [50]:
obs2, reward, done = guided.step(0)
print_node(guided)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

16353303012891903 NodeType.QUESTION
Are you traveling alone or with your family?
0 Alone
1 With family
0.01 False
0


In [51]:
obs2, reward, done = guided.step(1)
print_node(guided)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

16370483570980906 NodeType.INFO
Business traveler can receive reimbursement for business related expenses by submitting the following form.
0.03 False
0


In [52]:
obs2, reward, done = guided.step(0)
print_node(guided)
print(reward, done)
print(guided.last_valid_skip_transition_idx)

16370483570980906 NodeType.INFO
Business traveler can receive reimbursement for business related expenses by submitting the following form.
0.01 False
0


In [53]:
for line in guided.episode_log:
    print(line)

21142169-0$ MODE: GUIDED
21142169-1$ ======== RESET =========
21142169-1$ GOAL: 16460422136134149 If you are traveling for more than 4 weeks and need help finding accommodations, please contact the 
21142169-1$ CONSTRAINTS:
21142169-1$ 0) Variable TRIP_LENGTH' (TIMESPAN), constraints: 
 - > 672.0
21142169-1$ INITIAL UTTERANCE: Experimental term
21142169-1$ -> SKIPPED TO CORRECT NODE
21142169-1$ -> TURN REWARD: 3.0
21142169-1$ -> USER UTTERANCE: 
21142169-1$ TO NODE: userInputNode - 16370487978253782 - How long will your research semester last?  
21142169-1$ -> VAR NAME: TRIP_LENGTH, VALUE: 4642
21142169-1$ -> VAR NAME: TRIP_LENGTH, VALUE: 4642
21142169-1$ -> USER UTTERANCE: 4642
21142169-1$ TO NODE: userInputNode - 16370487978253782 - How long will your research semester last?  
21142169-1$ -> SKIPPED TO CORRECT NODE
21142169-1$ -> AUTO SKIP LOGIC NODE: SUCCESS 16460422136134149
21142169-1$ -> TURN REWARD: 3.0
21142169-1$ -> USER UTTERANCE: 
21142169-1$ TO NODE: infoNode - 164604221361